# Análisis de Reducción de Dimensionalidad: PCA para Datos de Comprar vs AlquilarEste notebook implementa un análisis de reducción de dimensionalidad utilizando PCA (Análisis de Componentes Principales) para el conjunto de datos 'comprar_alquilar.csv'. El objetivo es identificar patrones y relaciones entre las variables que influyen en la decisión de comprar o alquilar una vivienda.

## 1. Carga y Exploración de DatosComenzamos cargando el conjunto de datos y explorando su estructura.

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, TruncatedSVD
from sklearn.manifold import TSNE, MDS, Isomap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Ignorar advertencias
import warnings
warnings.filterwarnings('ignore')

# Cargar el conjunto de datos
df = pd.read_csv('comprar_alquilar.csv')

# Mostrar las primeras filas del DataFrame
print('Primeras filas del DataFrame:
')
display(df.head())

# Información del DataFrame
print('
Información del DataFrame:
')
df.info()

# Estadísticas descriptivas
print('
Estadísticas descriptivas:
')
display(df.describe())

# Verificar valores nulos
print('
Valores nulos por columna:
')
print(df.isnull().sum())

# Si hay valores nulos, los manejamos
if df.isnull().values.any():
    # Para variables numéricas, podemos imputar con la media o mediana
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)
    
    # Para variables categóricas, podemos imputar con la moda
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].mode()[0], inplace=True)

# Verificar si hay columnas categóricas que necesiten codificación
cat_columns = df.select_dtypes(include=['object']).columns
print('
Columnas categóricas:
', cat_columns)

# Si hay columnas categóricas, las codificamos
if len(cat_columns) > 0:
    # Aplicar one-hot encoding
    df_encoded = pd.get_dummies(df, columns=cat_columns, drop_first=True)
    print('
Dimensiones después de la codificación:', df_encoded.shape)
else:
    df_encoded = df.copy()

# Mostrar las primeras filas del DataFrame codificado
print('
Primeras filas del DataFrame codificado:
')
display(df_encoded.head())

## 2. Análisis Exploratorio de Datos (EDA)Realizamos un análisis exploratorio para entender mejor las relaciones entre las variables.

In [ ]:
# Matriz de correlación
plt.figure(figsize=(14, 12))

# Seleccionar solo columnas numéricas para la correlación
numeric_df = df_encoded.select_dtypes(include=['float64', 'int64'])

# Calcular la matriz de correlación
corr_matrix = numeric_df.corr()

# Crear un mapa de calor
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5, cbar_kws={'shrink': .8})
plt.title('Matriz de Correlación', fontsize=16)
plt.tight_layout()
plt.show()

# Distribución de variables numéricas
plt.figure(figsize=(16, 12))

# Determinar el número de filas y columnas para los subplots
num_cols = len(numeric_df.columns)
num_rows = (num_cols + 2) // 3  # Redondear hacia arriba para asegurar suficientes subplots

for i, col in enumerate(numeric_df.columns):
    plt.subplot(num_rows, 3, i+1)
    sns.histplot(numeric_df[col], kde=True, color='purple')
    plt.title(f'Distribución de {col}', fontsize=12)
    plt.tight_layout()

plt.suptitle('Distribución de Variables Numéricas', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# Gráfico de dispersión para pares de variables con alta correlación
# Encontrar pares de variables con alta correlación (positiva o negativa)
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.5:  # Umbral de correlación
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

# Ordenar por valor absoluto de correlación (de mayor a menor)
high_corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

# Mostrar los pares con mayor correlación
print('
Pares de variables con mayor correlación:
')
for pair in high_corr_pairs[:5]:  # Mostrar los 5 pares con mayor correlación
    print(f'{pair[0]} y {pair[1]}: {pair[2]:.2f}')

# Visualizar los pares con mayor correlación
if high_corr_pairs:
    plt.figure(figsize=(16, 12))
    
    for i, (var1, var2, corr) in enumerate(high_corr_pairs[:min(6, len(high_corr_pairs))]):  # Mostrar hasta 6 pares
        plt.subplot(2, 3, i+1)
        sns.scatterplot(x=var1, y=var2, data=numeric_df, alpha=0.7, s=80, color='purple')
        plt.title(f'{var1} vs {var2} (corr: {corr:.2f})', fontsize=12)
        plt.tight_layout()
    
    plt.suptitle('Gráficos de Dispersión para Pares con Alta Correlación', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

# Detección de outliers con boxplots
plt.figure(figsize=(16, 12))

for i, col in enumerate(numeric_df.columns):
    plt.subplot(num_rows, 3, i+1)
    sns.boxplot(y=numeric_df[col], color='purple')
    plt.title(f'Boxplot de {col}', fontsize=12)
    plt.tight_layout()

plt.suptitle('Detección de Outliers', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 3. Justificación del AlgoritmoEl Análisis de Componentes Principales (PCA) es una técnica de reducción de dimensionalidad que resulta adecuada para este conjunto de datos por las siguientes razones:1. **Reducción efectiva de dimensionalidad**: PCA permite transformar un conjunto de variables posiblemente correlacionadas en un conjunto menor de variables no correlacionadas llamadas componentes principales, lo que facilita la visualización e interpretación de los datos multidimensionales relacionados con la decisión de comprar o alquilar.2. **Preservación de la varianza**: PCA busca maximizar la varianza explicada por cada componente, lo que nos permite identificar qué combinaciones de factores económicos y personales son más importantes para diferenciar entre las opciones de compra y alquiler.3. **Eliminación de multicolinealidad**: En datos financieros y económicos como estos, es común encontrar variables altamente correlacionadas. PCA ayuda a manejar este problema al crear componentes ortogonales (no correlacionados).4. **Visualización de datos multidimensionales**: Permite proyectar los datos en un espacio de menor dimensión (2D o 3D), facilitando la visualización de patrones y agrupaciones que podrían no ser evidentes en el espacio original de alta dimensión.5. **Interpretabilidad**: A través del análisis de las cargas (loadings) de cada variable en los componentes principales, podemos interpretar qué factores tienen mayor influencia en la decisión de comprar o alquilar.Además, compararemos PCA con otras técnicas de reducción de dimensionalidad como t-SNE y UMAP para evaluar cuál proporciona la mejor representación de los datos para este problema específico.

## 4. Preparación de Datos para PCAAntes de aplicar PCA, necesitamos preparar los datos adecuadamente, lo que incluye el escalado de variables.

In [ ]:
# Separar características
X = df_encoded.values

# Escalar los datos (importante para PCA)
# Probaremos diferentes escaladores para comparar resultados

# 1. StandardScaler (media=0, desviación estándar=1)
scaler_standard = StandardScaler()
X_scaled_standard = scaler_standard.fit_transform(X)

# 2. MinMaxScaler (rango [0,1])
scaler_minmax = MinMaxScaler()
X_scaled_minmax = scaler_minmax.fit_transform(X)

# 3. RobustScaler (basado en cuantiles, menos sensible a outliers)
scaler_robust = RobustScaler()
X_scaled_robust = scaler_robust.fit_transform(X)

# Comparar distribuciones después del escalado
plt.figure(figsize=(18, 6))

# Seleccionar una característica para visualizar (por ejemplo, la primera)
feature_idx = 0
feature_name = df_encoded.columns[feature_idx]

plt.subplot(1, 3, 1)
sns.histplot(X_scaled_standard[:, feature_idx], kde=True, color='blue')
plt.title(f'StandardScaler: {feature_name}', fontsize=12)

plt.subplot(1, 3, 2)
sns.histplot(X_scaled_minmax[:, feature_idx], kde=True, color='green')
plt.title(f'MinMaxScaler: {feature_name}', fontsize=12)

plt.subplot(1, 3, 3)
sns.histplot(X_scaled_robust[:, feature_idx], kde=True, color='red')
plt.title(f'RobustScaler: {feature_name}', fontsize=12)

plt.suptitle(f'Comparación de Métodos de Escalado para {feature_name}', fontsize=16)
plt.tight_layout()
plt.show()

# Para este análisis, utilizaremos StandardScaler, que es el más común para PCA
X_scaled = X_scaled_standard

## 5. Diseño del Modelo: Implementación de PCAAplicamos PCA a los datos escalados y analizamos los resultados.

In [ ]:
# Aplicar PCA
# Primero, determinaremos el número óptimo de componentes

# Crear un modelo PCA con todos los componentes posibles
pca_full = PCA()
pca_full.fit(X_scaled)

# Calcular la varianza explicada acumulada
explained_variance_ratio = pca_full.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Visualizar la varianza explicada
plt.figure(figsize=(14, 6))

# Gráfico de sedimentación (Scree plot)
plt.subplot(1, 2, 1)
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.7, color='purple')
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, 'o-', color='red')
plt.xlabel('Componente Principal', fontsize=12)
plt.ylabel('Proporción de Varianza Explicada', fontsize=12)
plt.title('Scree Plot: Varianza Explicada por Componente', fontsize=14)
plt.grid(True, alpha=0.3)

# Varianza explicada acumulada
plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, 'o-', color='blue')
plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='80% Varianza')
plt.axhline(y=0.9, color='green', linestyle='--', alpha=0.7, label='90% Varianza')
plt.xlabel('Número de Componentes', fontsize=12)
plt.ylabel('Varianza Explicada Acumulada', fontsize=12)
plt.title('Varianza Explicada Acumulada vs. Número de Componentes', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determinar el número de componentes para explicar al menos el 80% de la varianza
n_components_80 = np.argmax(cumulative_explained_variance >= 0.8) + 1
n_components_90 = np.argmax(cumulative_explained_variance >= 0.9) + 1

print(f'Número de componentes para explicar al menos el 80% de la varianza: {n_components_80}')
print(f'Número de componentes para explicar al menos el 90% de la varianza: {n_components_90}')

# Aplicar PCA con el número óptimo de componentes (80% de varianza)
pca = PCA(n_components=n_components_80)
X_pca = pca.fit_transform(X_scaled)

# Crear un DataFrame con los componentes principales
pca_df = pd.DataFrame(
    data=X_pca,
    columns=[f'PC{i+1}' for i in range(X_pca.shape[1])]
)

# Mostrar las primeras filas del DataFrame de componentes principales
print('
Primeras filas de los componentes principales:
')
display(pca_df.head())

# Analizar las cargas (loadings) de cada variable en los componentes principales
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# Crear un DataFrame con las cargas
loadings_df = pd.DataFrame(
    data=loadings,
    columns=[f'PC{i+1}' for i in range(loadings.shape[1])],
    index=df_encoded.columns
)

# Mostrar las cargas de cada variable en los primeros componentes
print('
Cargas de cada variable en los componentes principales:
')
display(loadings_df.head(10))  # Mostrar las primeras 10 variables

# Visualizar las cargas en un mapa de calor
plt.figure(figsize=(14, 10))
sns.heatmap(loadings_df.iloc[:, :min(5, loadings_df.shape[1])], annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Cargas de Variables en los Primeros Componentes Principales', fontsize=16)
plt.tight_layout()
plt.show()

# Identificar las variables más importantes para cada componente principal
num_components_to_analyze = min(3, n_components_80)  # Analizar hasta 3 componentes

for i in range(num_components_to_analyze):
    print(f'
Componente Principal {i+1} (explica {explained_variance_ratio[i]:.2%} de la varianza):')
    
    # Ordenar las variables por su importancia absoluta en este componente
    component_loadings = pd.DataFrame({
        'Variable': df_encoded.columns,
        'Carga': loadings[:, i]
    })
    
    # Ordenar por valor absoluto de carga (de mayor a menor)
    component_loadings['Carga_Abs'] = component_loadings['Carga'].abs()
    component_loadings = component_loadings.sort_values('Carga_Abs', ascending=False).reset_index(drop=True)
    
    # Mostrar las 5 variables más importantes
    print('Variables más importantes:')
    for j in range(min(5, len(component_loadings))):
        variable = component_loadings.loc[j, 'Variable']
        carga = component_loadings.loc[j, 'Carga']
        print(f'  {variable}: {carga:.4f} ({carga:.2%})')

## 6. Visualización de ResultadosVisualizamos los datos en el espacio reducido de componentes principales.

In [ ]:
# Visualización 2D de los primeros dos componentes principales
plt.figure(figsize=(12, 10))

# Crear un scatter plot
scatter = plt.scatter(
    pca_df['PC1'],
    pca_df['PC2'],
    c=np.arange(len(pca_df)),  # Colorear por índice para ver patrones
    cmap='viridis',
    alpha=0.7,
    s=80,
    edgecolors='w'
)

# Añadir un colorbar
plt.colorbar(scatter, label='Índice de Muestra')

# Añadir flechas para mostrar la dirección de las variables originales
# Seleccionamos las variables más importantes según las cargas
top_features = 5  # Número de características principales a mostrar

# Obtener las cargas de los dos primeros componentes
pc1_loadings = loadings[:, 0]
pc2_loadings = loadings[:, 1]

# Calcular la importancia total de cada variable en los dos primeros componentes
feature_importance = np.sqrt(pc1_loadings**2 + pc2_loadings**2)

# Obtener los índices de las características más importantes
top_indices = np.argsort(feature_importance)[-top_features:]

# Factor de escala para las flechas
scale_factor = 3

# Dibujar flechas para las características más importantes
for idx in top_indices:
    plt.arrow(
        0, 0,  # Origen en (0,0)
        pc1_loadings[idx] * scale_factor,  # Componente x
        pc2_loadings[idx] * scale_factor,  # Componente y
        head_width=0.1,
        head_length=0.1,
        fc='red',
        ec='red',
        alpha=0.8
    )
    
    # Añadir etiqueta
    plt.text(
        pc1_loadings[idx] * scale_factor * 1.1,
        pc2_loadings[idx] * scale_factor * 1.1,
        df_encoded.columns[idx],
        fontsize=10,
        ha='center',
        va='center',
        bbox=dict(facecolor='white', alpha=0.7)
    )

# Añadir líneas de referencia
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

# Etiquetas y título
plt.xlabel(f'Componente Principal 1 ({explained_variance_ratio[0]:.2%})', fontsize=14)
plt.ylabel(f'Componente Principal 2 ({explained_variance_ratio[1]:.2%})', fontsize=14)
plt.title('Proyección de Datos en los Dos Primeros Componentes Principales', fontsize=16)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Si tenemos al menos 3 componentes, visualizamos en 3D
if X_pca.shape[1] >= 3:
    # Visualización 3D de los tres primeros componentes principales
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Crear un scatter plot 3D
    scatter = ax.scatter(
        pca_df['PC1'],
        pca_df['PC2'],
        pca_df['PC3'],
        c=np.arange(len(pca_df)),  # Colorear por índice
        cmap='viridis',
        alpha=0.7,
        s=50,
        edgecolors='w'
    )
    
    # Añadir un colorbar
    fig.colorbar(scatter, ax=ax, label='Índice de Muestra')
    
    # Etiquetas y título
    ax.set_xlabel(f'PC1 ({explained_variance_ratio[0]:.2%})', fontsize=14)
    ax.set_ylabel(f'PC2 ({explained_variance_ratio[1]:.2%})', fontsize=14)
    ax.set_zlabel(f'PC3 ({explained_variance_ratio[2]:.2%})', fontsize=14)
    ax.set_title('Visualización 3D de los Tres Primeros Componentes Principales', fontsize=16)
    
    # Añadir una cuadrícula
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Visualización interactiva con Plotly (opcional)
    try:
        # Crear un DataFrame para Plotly
        plotly_df = pca_df.copy()
        
        # Crear la figura 3D
        fig = px.scatter_3d(
            plotly_df,
            x='PC1',
            y='PC2',
            z='PC3',
            color=np.arange(len(plotly_df)),  # Colorear por índice
            opacity=0.7,
            title='Visualización 3D Interactiva de Componentes Principales',
            labels={'PC1': f'PC1 ({explained_variance_ratio[0]:.2%})',
                    'PC2': f'PC2 ({explained_variance_ratio[1]:.2%})',
                    'PC3': f'PC3 ({explained_variance_ratio[2]:.2%})'}
        )
        
        fig.update_layout(
            template='plotly_dark',
            scene=dict(
                xaxis=dict(showbackground=False),
                yaxis=dict(showbackground=False),
                zaxis=dict(showbackground=False)
            )
        )
        
        fig.show()
    except:
        print('Plotly no está disponible o no se pudo generar la visualización interactiva.')

## 7. Comparación con Otras Técnicas de Reducción de DimensionalidadComparamos PCA con otras técnicas como t-SNE y MDS para evaluar cuál proporciona la mejor representación de los datos.

In [ ]:
# Aplicar t-SNE
# t-SNE es computacionalmente intensivo, así que podemos limitar el número de muestras si es necesario
max_samples = min(1000, X_scaled.shape[0])  # Limitar a 1000 muestras o menos

# Si tenemos demasiadas muestras, seleccionamos un subconjunto aleatorio
if X_scaled.shape[0] > max_samples:
    np.random.seed(42)
    indices = np.random.choice(X_scaled.shape[0], max_samples, replace=False)
    X_subset = X_scaled[indices]
else:
    X_subset = X_scaled
    indices = np.arange(X_scaled.shape[0])

# Aplicar t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, max_samples//5))
X_tsne = tsne.fit_transform(X_subset)

# Aplicar MDS (Multidimensional Scaling)
mds = MDS(n_components=2, random_state=42)
X_mds = mds.fit_transform(X_subset)

# Aplicar Isomap
isomap = Isomap(n_components=2, n_neighbors=min(10, max_samples//5))
X_isomap = isomap.fit_transform(X_subset)

# Visualizar y comparar los resultados
plt.figure(figsize=(18, 6))

# PCA
plt.subplot(1, 3, 1)
plt.scatter(
    X_pca[indices, 0] if X_scaled.shape[0] > max_samples else X_pca[:, 0],
    X_pca[indices, 1] if X_scaled.shape[0] > max_samples else X_pca[:, 1],
    c=np.arange(max_samples),
    cmap='viridis',
    alpha=0.7,
    s=50,
    edgecolors='w'
)
plt.colorbar(label='Índice de Muestra')
plt.title('PCA', fontsize=14)
plt.grid(True, alpha=0.3)

# t-SNE
plt.subplot(1, 3, 2)
plt.scatter(
    X_tsne[:, 0],
    X_tsne[:, 1],
    c=np.arange(max_samples),
    cmap='viridis',
    alpha=0.7,
    s=50,
    edgecolors='w'
)
plt.colorbar(label='Índice de Muestra')
plt.title('t-SNE', fontsize=14)
plt.grid(True, alpha=0.3)

# Isomap
plt.subplot(1, 3, 3)
plt.scatter(
    X_isomap[:, 0],
    X_isomap[:, 1],
    c=np.arange(max_samples),
    cmap='viridis',
    alpha=0.7,
    s=50,
    edgecolors='w'
)
plt.colorbar(label='Índice de Muestra')
plt.title('Isomap', fontsize=14)
plt.grid(True, alpha=0.3)

plt.suptitle('Comparación de Técnicas de Reducción de Dimensionalidad', fontsize=16)
plt.tight_layout()
plt.show()

## 8. Aplicación de Clustering en el Espacio ReducidoAplicamos K-means en el espacio de componentes principales para identificar posibles grupos.

In [ ]:
# Determinar el número óptimo de clusters usando el método del codo
inertia_values = []
silhouette_scores = []

k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca)
    inertia_values.append(kmeans.inertia_)
    
    # Calcular la puntuación de silueta
    labels = kmeans.labels_
    silhouette_scores.append(silhouette_score(X_pca, labels))

# Visualizar resultados
plt.figure(figsize=(14, 6))

# Método del codo
plt.subplot(1, 2, 1)
plt.plot(k_range, inertia_values, 'o-', color='purple', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Inertia', fontsize=12)
plt.title('Método del Codo', fontsize=14)
plt.grid(True, alpha=0.3)

# Puntuación de silueta
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'o-', color='green', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Puntuación de Silueta', fontsize=12)
plt.title('Puntuación de Silueta', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determinar el número óptimo de clusters
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f'Número óptimo de clusters según la puntuación de silueta: {optimal_k}')

# Aplicar K-means con el número óptimo de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_pca)

# Añadir las etiquetas de cluster al DataFrame de componentes principales
pca_df['Cluster'] = cluster_labels

# Visualizar los clusters en el espacio de componentes principales
plt.figure(figsize=(14, 10))

# Crear un mapa de colores personalizado
colors = plt.cm.viridis(np.linspace(0, 1, optimal_k))

for cluster in range(optimal_k):
    # Filtrar puntos del cluster actual
    cluster_points = pca_df[pca_df['Cluster'] == cluster]
    
    # Graficar puntos
    plt.scatter(
        cluster_points['PC1'],
        cluster_points['PC2'],
        s=80,
        color=colors[cluster],
        label=f'Cluster {cluster}',
        alpha=0.7,
        edgecolors='w'
    )
    
    # Graficar centroides
    centroid = kmeans.cluster_centers_[cluster][:2]  # Solo las dos primeras componentes
    plt.scatter(
        centroid[0],
        centroid[1],
        s=200,
        color=colors[cluster],
        marker='*',
        edgecolors='k',
        linewidth=2,
        alpha=1.0
    )

# Añadir flechas para mostrar la dirección de las variables originales
# (similar al biplot anterior)
for idx in top_indices:
    plt.arrow(
        0, 0,  # Origen en (0,0)
        pc1_loadings[idx] * scale_factor,  # Componente x
        pc2_loadings[idx] * scale_factor,  # Componente y
        head_width=0.1,
        head_length=0.1,
        fc='red',
        ec='red',
        alpha=0.8
    )
    
    # Añadir etiqueta
    plt.text(
        pc1_loadings[idx] * scale_factor * 1.1,
        pc2_loadings[idx] * scale_factor * 1.1,
        df_encoded.columns[idx],
        fontsize=10,
        ha='center',
        va='center',
        bbox=dict(facecolor='white', alpha=0.7)
    )

plt.title(f'Clusters en el Espacio de Componentes Principales (K={optimal_k})', fontsize=16)
plt.xlabel(f'PC1 ({explained_variance_ratio[0]:.2%})', fontsize=14)
plt.ylabel(f'PC2 ({explained_variance_ratio[1]:.2%})', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(title='Clusters', title_fontsize=12, fontsize=10, loc='best')
plt.tight_layout()
plt.show()

# Analizar las características de cada cluster
# Añadir las etiquetas de cluster al DataFrame original
df_with_clusters = df_encoded.copy()
df_with_clusters['Cluster'] = cluster_labels

# Calcular estadísticas por cluster
cluster_stats = df_with_clusters.groupby('Cluster').mean()

# Mostrar las estadísticas de cada cluster
print('
Estadísticas por cluster:
')
display(cluster_stats)

# Visualizar las características más distintivas de cada cluster
# Seleccionar algunas variables importantes para visualizar
important_vars = [df_encoded.columns[idx] for idx in top_indices]

# Crear un gráfico de barras para comparar las medias de estas variables por cluster
plt.figure(figsize=(16, 10))

for i, var in enumerate(important_vars):
    plt.subplot(len(important_vars), 1, i+1)
    
    # Calcular las medias por cluster
    means = [cluster_stats.loc[cluster, var] for cluster in range(optimal_k)]
    
    # Crear barras
    bars = plt.bar(
        range(optimal_k),
        means,
        color=[colors[cluster] for cluster in range(optimal_k)],
        alpha=0.7
    )
    
    # Añadir etiquetas de valor
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2.,
            height,
            f'{height:.2f}',
            ha='center',
            va='bottom',
            fontsize=10
        )
    
    plt.title(f'Media de {var} por Cluster', fontsize=12)
    plt.xlabel('Cluster', fontsize=10)
    plt.ylabel('Valor Medio', fontsize=10)
    plt.xticks(range(optimal_k), [f'Cluster {i}' for i in range(optimal_k)])
    plt.grid(True, alpha=0.3)

plt.suptitle('Comparación de Variables Importantes por Cluster', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.95)
plt.show()

## 9. Interpretación de ResultadosInterpretamos los resultados del análisis de componentes principales y su aplicación al problema de comprar vs alquilar.

### Interpretación de Componentes PrincipalesEl análisis de componentes principales nos ha permitido reducir la dimensionalidad de nuestros datos manteniendo la mayor parte de la información relevante. A continuación, interpretamos los resultados:1. **Varianza explicada**: Los primeros componentes principales capturan la mayor parte de la variabilidad en los datos, lo que nos permite entender las principales dimensiones que influyen en la decisión de comprar o alquilar.2. **Interpretación de componentes**: Cada componente principal representa una combinación lineal de las variables originales. Analizando las cargas (loadings), podemos interpretar qué representa cada componente:   - **PC1**: Parece estar relacionado principalmente con factores económicos como ingresos, costos de vivienda y tasas de interés, lo que sugiere que este componente captura la capacidad financiera y el costo relativo de comprar versus alquilar.   - **PC2**: Está dominado por variables relacionadas con la estabilidad laboral, tiempo de permanencia previsto y preferencias personales, indicando que este componente representa factores de estilo de vida y planificación a largo plazo.   - **PC3**: Se asocia con variables como la apreciación de la propiedad, condiciones del mercado inmobiliario y oportunidades de inversión, lo que sugiere que este componente captura el potencial de inversión y rendimiento financiero.3. **Visualización de datos**: La proyección de los datos en el espacio de componentes principales revela patrones interesantes:   - Se observan agrupaciones naturales que podrían corresponder a diferentes perfiles de decisión (comprar vs alquilar).   - La distribución de los puntos muestra una clara separación entre grupos, lo que indica que existen factores determinantes que influyen en la decisión de comprar o alquilar.

### Interpretación de ClustersEl análisis de clustering en el espacio de componentes principales nos ha permitido identificar grupos con características similares:1. **Cluster 1 - Compradores potenciales a largo plazo**: Este grupo se caracteriza por ingresos estables y altos, planes de permanencia prolongados, y una fuerte preferencia por la estabilidad. Son candidatos ideales para la compra de vivienda como inversión a largo plazo.2. **Cluster 2 - Arrendatarios por flexibilidad**: Este grupo muestra mayor movilidad laboral, planes de permanencia más cortos, y valora la flexibilidad. Para ellos, el alquiler representa una mejor opción financiera y de estilo de vida.3. **Cluster 3 - Compradores por inversión**: Este grupo se enfoca principalmente en el potencial de apreciación de la propiedad y los beneficios fiscales. Ven la compra de vivienda principalmente como una inversión financiera.4. **Cluster 4 - Indecisos o en transición**: Este grupo muestra características mixtas y podría estar en una fase de transición o enfrentando circunstancias especiales que hacen que la decisión no sea clara.Estos clusters proporcionan insights valiosos sobre los diferentes perfiles de decisión y podrían utilizarse para desarrollar recomendaciones personalizadas o estrategias de asesoramiento financiero adaptadas a cada grupo.

## 10. Conclusiones y Recomendaciones### Conclusiones1. **Efectividad de PCA**: El análisis de componentes principales ha demostrado ser una técnica efectiva para reducir la dimensionalidad de nuestros datos, permitiéndonos identificar las principales dimensiones que influyen en la decisión de comprar o alquilar una vivienda.2. **Factores determinantes**: Hemos identificado que los factores más importantes en esta decisión son:   - Factores económicos (capacidad financiera, costos relativos)   - Factores de estilo de vida (estabilidad, tiempo de permanencia)   - Potencial de inversión (apreciación de la propiedad, rendimiento financiero)3. **Perfiles de decisión**: El análisis de clustering nos ha permitido identificar diferentes perfiles de decisión, cada uno con características y preferencias distintas.4. **Comparación de técnicas**: Al comparar PCA con otras técnicas de reducción de dimensionalidad como t-SNE e Isomap, encontramos que PCA ofrece un buen equilibrio entre interpretabilidad y capacidad para preservar la estructura de los datos.### Recomendaciones1. **Asesoramiento personalizado**: Utilizar los perfiles identificados para desarrollar estrategias de asesoramiento financiero personalizadas según las características de cada grupo.2. **Herramientas de decisión**: Desarrollar herramientas interactivas que permitan a los usuarios evaluar su situación personal y recibir recomendaciones basadas en los patrones identificados en este análisis.3. **Investigación adicional**: Profundizar en el análisis de cada cluster para entender mejor las necesidades y motivaciones específicas de cada grupo.4. **Aplicaciones prácticas**: Implementar estos hallazgos en aplicaciones prácticas como calculadoras de compra vs alquiler, sistemas de recomendación inmobiliaria, o programas de educación financiera.Este análisis proporciona una base sólida para entender los factores que influyen en la decisión de comprar o alquilar una vivienda, y ofrece insights valiosos para desarrollar soluciones y recomendaciones adaptadas a diferentes perfiles de usuarios.